# Вариант 1
**Цель:** Контроль дисперсии весов.

- Разработайте архитектуру, которая будет за 5 слоев приводить тензор к размерности (1, 512, 1, 1).
- Условие: дисперсия весов третьего слоя должна быть в два раза больше, чем у второго и четвертого.
- Эксперимент: Проверьте влияние увеличенной дисперсии третьего слоя на распределение активаций после GAP и визуализируйте их гистограмму.

In [7]:
import torch.nn as nn
import torch
import numpy as np

In [ ]:
class MyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.convLayer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.convLayer2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, stride=2, padding=1)
        self.convLayer3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=2, padding=1)
        self.convLayer4 = nn.Conv2d(in_channels=32, out_channels=256, kernel_size=3, stride=2, padding=1)
        self.convLayer5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=2, padding=1)
        #self.avgPool = nn.AvgPool2d(kernel_size=2, stride=2)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        #x = self.input(x)
        print('conv1 var', np.round(self.convLayer1.weight.data.var().item(),6))
        x = self.convLayer1(x)
        x = self.activation(x)
        print('conv2 var', np.round(self.convLayer2.weight.data.var().item(),6))
        x = self.convLayer2(x)
        x = self.activation(x)
        print('conv3 var', np.round(self.convLayer3.weight.data.var().item(),6))
        x = self.convLayer3(x)
        x = self.activation(x)
        print('conv4 var', np.round(self.convLayer4.weight.data.var().item(),6))
        x = self.convLayer4(x)
        x = self.activation(x)
        print('conv5 var', np.round(self.convLayer5.weight.data.var().item(),6))
        x = self.convLayer5(x)
        x = self.activation(x)
        #x = self.avgPool(x)
        return x

model = MyNet()
# batch=1, channel=3, size=16x16
output = model(torch.randn(1,3,16,16))
output.shape

conv1 var 0.011902
conv2 var 0.001169
conv3 var 0.00236
conv4 var 0.001165
conv5 var 0.000145


torch.Size([1, 512, 1, 1])

In [ ]:
'''
print('conv3 var', self.convLayer3.weight.data.var().item())
var2 = self.convLayer2.weight.data.var().item()
var3 = self.convLayer3.weight.data.var().item()
var4 = self.convLayer4.weight.data.var().item()
'''